# Statement
In this notebook, I’m building a simple feed-forward neural network from scratch using PyTorch to classify the Breast Cancer dataset. The goal is to predict, based on given features, whether a patient has cancer or not.

The primary objective here is not to achieve a highly accurate model, but rather to understand and implement the complete neural network pipeline in PyTorch from the ground up. This serves as my first step in learning PyTorch before moving on to more advanced architectures in future projects.

I’m following the PyTorch learning series from CampusX YouTube channel as a reference while building this project. "https://www.youtube.com/@campusx-official"

In [1]:
 # import required packages
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

## Load Data

In [2]:
 df = pd.read_csv("https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv")
df

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,...,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,...,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,...,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,...,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,...,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,926424,M,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,...,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115,NaN
565,926682,M,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,...,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637,NaN
566,926954,M,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,...,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820,NaN
567,927241,M,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,...,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400,NaN


In [3]:
 # Get the shape of dataset
 df.shape

(569, 33)

In [4]:
# Dropping irrelevant columns from data
df.drop(columns = ['id', 'Unnamed: 32'], inplace=True)
df

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,...,25.380,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,...,24.990,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,...,23.570,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,...,14.910,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,...,22.540,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,M,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,...,25.450,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115
565,M,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,...,23.690,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637
566,M,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,...,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820
567,M,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,...,25.740,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400


## Train Test Split

To train and evaluate the model 80% of the data will be used for training and 20% of the data will be used for evaluation.

In [5]:
x_train, x_test, y_train, y_test = train_test_split(df.iloc[:, 1:], df.iloc[:, 0], test_size = 0.2)

## Scaling

As differnt columns features were of different scales, scale all features on same level.



In [6]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [7]:
x_train

array([[-0.33494767, -1.45022225, -0.40006416, ..., -0.92338844,
        -0.94557001, -0.80631197],
       [-0.09167314, -0.35309295, -0.14178969, ..., -0.82373881,
        -0.8805121 , -0.75268981],
       [ 0.59235759, -0.35996433,  0.64091046, ...,  1.10226113,
         2.16610258,  2.12665737],
       ...,
       [-1.02470251,  0.09354632, -1.02896042, ..., -0.71432261,
        -0.03475917, -0.35512758],
       [-0.10598341, -0.77453719, -0.15339753, ..., -0.6760193 ,
         0.66342336, -0.38058456],
       [-0.45801596, -1.51664553, -0.52692128, ..., -1.33327957,
        -0.99158659, -0.74510688]])

In [8]:
y_train

,diagnosis
287,B
149,B
34,M
568,B
449,M
...,...
10,M
334,B
245,B
374,B


## Label Encoding

The labels are binary encoded, we can't use them directly. We need to make numberical representation of our labels.

In [9]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

In [10]:
y_train

array([0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1,
       0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0,
       0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1,
       0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0,
       1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1,

## Conversion from Numpy array to pyTorch tensors

In [11]:
x_train_tensor = torch.from_numpy(x_train)
x_test_tensor = torch.from_numpy(x_test)
y_train_tensor = torch.from_numpy(y_train)
y_test_tensor = torch.from_numpy(y_test)

In [12]:
# Get shape of training tensor
x_train_tensor.shape

torch.Size([455, 30])

In [13]:
y_train_tensor.shape

torch.Size([455])

## Defining the Model

In [14]:
class MySimpleNN():

  def __init__(self, X):
    self.weights = torch.rand(X.shape[1], 1, dtype = torch.float64, requires_grad = True)
    self.bias = torch.zeros(1, dtype = torch.float64, requires_grad = True)

  def forward(self, X):
    z = torch.matmul(X, self.weights) + self.bias
    y_pred = torch.sigmoid(z)
    return y_pred

  def loss_function(self, y_pred, y):
    # Clamp prediction to avoid log(0)
    epsilon = 1e-7
    y_pred = torch.clamp(y_pred, epsilon, 1 - epsilon)

    # Calculate Loss
    loss = -(y_train_tensor * torch.log(y_pred) + (1- y_train_tensor) * torch.log(1 - y_pred)).mean()
    return loss

## Important Parameters

In [15]:
learning_rate = 0.1
epochs = 100

# Training Pipeline

In [16]:
# Create a mdoel object for defined class
model = MySimpleNN(x_train_tensor)

# define Training Loop
for epoch in range(epochs):
  # forward Pass
  y_pred = model.forward(x_train_tensor)
  # print(y_pred)

  # loss calculation
  loss = model.loss_function(y_pred, y_train_tensor)
  # backward pass
  loss.backward()

  # Parameter Update
  with torch.no_grad():
    model.weights -= learning_rate * model.weights.grad
    model.bias -= learning_rate * model.bias.grad

  # For next iteration reset gradients value to 0
  # zero gradients
  model.weights.grad.zero_()
  model.bias.grad.zero_()

  print(f"Epoch: {epoch+1}, loss: {loss.item()}")


Epoch: 1, loss: 3.8112056163975887
Epoch: 2, loss: 3.6910622865585
Epoch: 3, loss: 3.5657880548317724
Epoch: 4, loss: 3.4351602307160864
Epoch: 5, loss: 3.3002186067175527
Epoch: 6, loss: 3.1629017638215906
Epoch: 7, loss: 3.0182821291832314
Epoch: 8, loss: 2.865738041140953
Epoch: 9, loss: 2.7097427358182853
Epoch: 10, loss: 2.546789018299206
Epoch: 11, loss: 2.3759270209413215
Epoch: 12, loss: 2.209673606200215
Epoch: 13, loss: 2.046807952144249
Epoch: 14, loss: 1.8844347760643094
Epoch: 15, loss: 1.7256411874841842
Epoch: 16, loss: 1.5731289197081106
Epoch: 17, loss: 1.4302256264898423
Epoch: 18, loss: 1.3021809221820828
Epoch: 19, loss: 1.190694457862366
Epoch: 20, loss: 1.0968279890837787
Epoch: 21, loss: 1.0206508167137553
Epoch: 22, loss: 0.9610452033818544
Epoch: 23, loss: 0.915808471659836
Epoch: 24, loss: 0.8820819268716534
Epoch: 25, loss: 0.8569418875934532
Epoch: 26, loss: 0.8378682748543399
Epoch: 27, loss: 0.8229422727562885
Epoch: 28, loss: 0.8108255368758888
Epoch: 29,

In [17]:
# Check model weights
print(model.weights)

tensor([[-0.1558],
        [ 0.1055],
        [ 0.3759],
        [ 0.3551],
        [ 0.0961],
        [-0.1144],
        [-0.2455],
        [-0.1206],
        [ 0.0313],
        [-0.1974],
        [ 0.3388],
        [ 0.0122],
        [-0.3522],
        [-0.2219],
        [ 0.1734],
        [ 0.3028],
        [ 0.3102],
        [ 0.1742],
        [-0.1099],
        [-0.3856],
        [-0.4396],
        [-0.1209],
        [ 0.3139],
        [ 0.3213],
        [ 0.0809],
        [-0.2129],
        [-0.3378],
        [-0.4021],
        [ 0.2762],
        [ 0.6137]], dtype=torch.float64, requires_grad=True)


## Evaluation


In [18]:
# Model evauluation
with torch.no_grad():
  y_pred = model.forward(x_test_tensor)
  y_pred = (y_pred > 0.8).float()
  accuracy = (y_pred == y_test_tensor).float().mean()
  print(f"Accuracy: {accuracy * 100:.2f} %")

Accuracy: 61.40 %


# Conclusion
Although the model accuracy is not good, the reason will be I've just added one single layer. By using multiple layers the performance can be incrased.